In [1]:
import logging


def init_logger(file_name, log_level=None):
    if not log_level:
        log_level = "INFO"
    logging.basicConfig()
    logger = logging.getLogger(file_name)
    logger.setLevel(getattr(logging, log_level))
    return logger

In [2]:
import awswrangler as wr
import boto3
import pandas as pd
from collections import defaultdict
import json
import numpy as np

In [3]:
logger = init_logger(__name__)

In [4]:
myProfile = 'jvalenzuela-tesis'
mySession = boto3.session.Session(profile_name=myProfile, region_name = 'us-east-1')
ssmClient = mySession.client('ssm', region_name = 'us-east-1')

In [5]:
rawBucket = ssmClient.get_parameter(Name='/Master/S3/CentralBucket')['Parameter']['Value']
stageBucket = ssmClient.get_parameter(Name='/Master/S3/StageBucket')['Parameter']['Value']
analyticsBucket = ssmClient.get_parameter(Name='/Master/S3/AnalyticsBucket')['Parameter']['Value']

In [6]:
event = {"dataset":"events"}
context = None

In [29]:
def lambda_handler(event, context): 
    try:
        dataset = event['dataset']
        analyticsDataset = f'an{dataset}'
        s3StagePath = f"s3://{stageBucket}/datalake/{dataset}/"
        s3AnalyticsPath = f"s3://{analyticsBucket}/datalake/{analyticsDataset}2/"
        
        print(f"Using tha following Stage bucket Path: {s3StagePath}")        
        transformDataframeEvents(s3StagePath, s3AnalyticsPath)
        
    except Exception as e: 
        print(f"Error on LambdaHandler has ocurred: {str(e)}")

In [32]:
def transformDataframeEvents(s3StagePath, s3AnalyticsPath):
    
    df = wr.s3.read_parquet(s3StagePath, dataset=True, boto3_session=mySession)
    
    df=df[(df['period']!=5)]
    df[['loc_x','loc_y']] = df.location.apply(pd.Series)
    df['loc_x']=df['loc_x'].replace("NaN", value=pd.np.nan).astype(float)
    df['loc_y']=df['loc_y'].replace("NaN", value=pd.np.nan).astype(float)
    df['loc_x'] = df['loc_x'].fillna(-1)
    df['loc_y'] = df['loc_y'].fillna(-1)
    
    #creacion de variables 
    df['shot.dangerous'] = np.where((df['loc_x'].astype(int)>=94) & (df['loc_y'].astype(int)>=25) & (df['loc_y'].astype(int)<=60), True, False)
    
    df_array = []
    #Agrupar por posession, quiero saber todas las maniobras en un shot. 
    for (cc), group_poss in df.groupby('possession'):

        if True in set(group_poss['shot.dangerous']):
          group_poss['possession.dangerous']= 'True'
          group_poss['possession']= cc
        else:
          group_poss['possession.dangerous']= 'False'
          group_poss['possession']= cc
        groupo=group_poss[(group_poss['possession.dangerous']=='True')]
        df_array.append(groupo)

    df_concat_events = pd.concat(df_array)
    
    print(f"Lenght of array : {len(df_array)}")
    
    wr.s3.to_parquet(
            df=df_concat_events,
            path= s3AnalyticsPath,
            dataset = True,
            mode = 'overwrite_partitions',
            partition_cols = ['match_id'],
            boto3_session=mySession
        )

In [33]:
lambda_handler(event, context)

Using tha following Stage bucket Path: s3://soccer-datalake-dev-us-east-1-829547505377-stage/datalake/events/


/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/2034134403.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['loc_x','loc_y']] = df.location.apply(pd.Series)
/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/2034134403.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['loc_x','loc_y']] = df.location.apply(pd.Series)
/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/2034134403.py:6: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['loc_x','loc_y']] = df.location.apply(pd.Series)
/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/2034134403.py:6: FutureWarning: The defau

Lenght of array : 198


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):


In [19]:
s3StagePath = f"s3://{stageBucket}/datalake/events/"
df = wr.s3.read_parquet(s3StagePath, dataset=True, boto3_session=mySession)
    
df=df[(df['period']!=5)]


In [23]:
df[['loc_x','loc_y']] = df.location.apply(pd.Series)
df['loc_x']=df['loc_x'].replace("NaN", value=pd.np.nan).astype(float)
df['loc_y']=df['loc_y'].replace("NaN", value=pd.np.nan).astype(float)
df['loc_x'] = df['loc_x'].fillna(-1)
df['loc_y'] = df['loc_y'].fillna(-1)
#creacion de variables 
df['shot.dangerous'] = np.where((df['loc_x'].astype(int)>=94) & (df['loc_y'].astype(int)>=25) & (df['loc_y'].astype(int)<=60), True, False)



/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/42238408.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['loc_x','loc_y']] = df.location.apply(pd.Series)
/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/42238408.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['loc_x','loc_y']] = df.location.apply(pd.Series)
/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/42238408.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df[['loc_x','loc_y']] = df.location.apply(pd.Series)
/var/folders/f4/xgxbfgyd78l0z48nck9l57zh0000gn/T/ipykernel_39717/42238408.py:1: FutureWarning: The default dtype

In [24]:
df_array = []
#Agrupar por posession, quiero saber todas las maniobras en un shot. 
for (cc), group_poss in df.groupby('possession'):

    if True in set(group_poss['shot.dangerous']):
      group_poss['possession.dangerous']= 'True'
      group_poss['possession']= cc
    else:
      group_poss['possession.dangerous']= 'False'
      group_poss['possession']= cc
    groupo=group_poss[(group_poss['possession.dangerous']=='True')]
    df_array.append(groupo)

df_concat_events = pd.concat(df_array)



In [27]:
s3AnalyticsPath = f"s3://{analyticsBucket}/datalake/anevents/"

In [28]:
print(f"Lenght of array : {len(df_array)}")

wr.s3.to_parquet(
        df=df_concat_events,
        path= s3AnalyticsPath,
        dataset = True,
        mode = 'overwrite_partitions',
        partition_cols = ['match_id'],
        boto3_session=mySession
    )

Lenght of array : 294


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):


KeyboardInterrupt: 

In [13]:
s3StagePath = f"s3://{stageBucket}/datalake/events/"

In [29]:
df = wr.s3.read_parquet(s3StagePath, dataset=True, boto3_session=mySession)

In [30]:
df=df[(df['period']!=5)]

In [ ]:
#df[['loc_x','loc_y']]=df["location"].apply(lambda x: pd.Series(str(x).strip('[]').split(",")))
df[['loc_x','loc_y']] = df.location.apply(pd.Series)
df['loc_x']=df['loc_x'].replace("NaN", value=pd.np.nan).astype(float)
df['loc_y']=df['loc_y'].replace("NaN", value=pd.np.nan).astype(float)
df['loc_x'] = df['loc_x'].fillna(-1)
df['loc_y'] = df['loc_y'].fillna(-1)

In [37]:
import numpy as np

In [38]:
#creacion de variables 
df['shot.dangerous'] = np.where((df['loc_x'].astype(int)>=94) & (df['loc_y'].astype(int)>=25) & (df['loc_y'].astype(int)<=60), True, False)

In [40]:
df_array = []
#Agrupar por posession, quiero saber todas las maniobras en un shot. 
for (cc), group_poss in df.groupby('possession'):

    if True in set(group_poss['shot.dangerous']):
      group_poss['possession.dangerous']= 'True'
      group_poss['possession']= cc
    else:
      group_poss['possession.dangerous']= 'False'
      group_poss['possession']= cc
    groupo=group_poss[(group_poss['possession.dangerous']=='True')]
    df_array.append(groupo)
    
df_concat_events = pd.concat(df_array)

In [44]:
s3AnalyticsPath = f"s3://{analyticsBucket}/datalake/anevents/"

In [45]:
wr.s3.to_parquet(
            df=df_concat_events,
            path= s3AnalyticsPath,
            dataset = True,
            mode = 'overwrite_partitions',
            partition_cols = ['filename'],
            boto3_session=mySession
        )

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/awswrangler/s3/_write_dataset.py:92: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for keys, subgroup in df.groupby(by=partition_cols, observed=True):


{'paths': ['s3://soccer-datalake-dev-us-east-1-829547505377-analytics/datalake/anevents/filename=15946/186d9671175c4259a4bfcc786c852843.snappy.parquet',
  's3://soccer-datalake-dev-us-east-1-829547505377-analytics/datalake/anevents/filename=15956/186d9671175c4259a4bfcc786c852843.snappy.parquet',
  's3://soccer-datalake-dev-us-east-1-829547505377-analytics/datalake/anevents/filename=15973/186d9671175c4259a4bfcc786c852843.snappy.parquet',
  's3://soccer-datalake-dev-us-east-1-829547505377-analytics/datalake/anevents/filename=15978/186d9671175c4259a4bfcc786c852843.snappy.parquet',
  's3://soccer-datalake-dev-us-east-1-829547505377-analytics/datalake/anevents/filename=15986/186d9671175c4259a4bfcc786c852843.snappy.parquet',
  's3://soccer-datalake-dev-us-east-1-829547505377-analytics/datalake/anevents/filename=15998/186d9671175c4259a4bfcc786c852843.snappy.parquet',
  's3://soccer-datalake-dev-us-east-1-829547505377-analytics/datalake/anevents/filename=16010/186d9671175c4259a4bfcc786c852843.

In [ ]:
df_array = []
for f_name in glob(eventsFiles + '\*.json'):
  f = open (f_name, "r", encoding="utf8")
  df = pd.json_normalize(json.loads(f.read()) 
              #record_path =['lineup'] 
              #meta=['team_id', 'team_name']
            )
  file_name = os.path.splitext(os.path.basename(f_name))[0]
  df['match_id']=file_name
  print(file_name)
  
  #Penales parte final
  df=df[(df['period']!=5)]
  
  df[['loc_x','loc_y']]=df["location"].apply(lambda x: pd.Series(str(x).strip('[]').split(",")))
  df['loc_x']=df['loc_x'].replace("NaN", value=pd.np.nan).astype(float)
  df['loc_y']=df['loc_y'].replace("NaN", value=pd.np.nan).astype(float)
  df['loc_x'] = df['loc_x'].fillna(-1)
  df['loc_y'] = df['loc_y'].fillna(-1)

  #creacion de variables 
  df['shot.dangerous'] = np.where((df['loc_x'].astype(int)>=94) & (df['loc_y'].astype(int)>=25) & (df['loc_y'].astype(int)<=60), True, False)
  
  #Agrupar por posession, quiero saber todas las maniobras en un shot. 
  for (cc), group_poss in df.groupby('possession'):

    if True in set(group_poss['shot.dangerous']):
      group_poss['possession.dangerous']= 'True'
      group_poss['possession']= cc
    else:
      group_poss['possession.dangerous']= 'False'
      group_poss['possession']= cc
    groupo=group_poss[(group_poss['possession.dangerous']=='True')]
    df_array.append(groupo)
    
df_concat_events = pd.concat(df_array)
print('Done')